In [ ]:
%load_ext autoreload
%autoreload 2

In [24]:
from time import time
from datasets import load_dataset
from perfcounters import PerfCounters

In [10]:
train = load_dataset("rotten_tomatoes", split="train")
test = load_dataset("rotten_tomatoes", split="test")

train_texts = [t['text'] for t in train]
test_texts = [t['text'] for t in test]
print(f"train_texts: {len(train_texts)} - test_texts: {len(test_texts)}")

train_texts: 8530 - test_texts: 1066


In [12]:
from unisim.config import use_tf_backend
use_tf_backend()

In [14]:

from unisim import ExactUniSim, ApproxUniSim
usim = ExactUniSim(store_data=True, use_tf_knn=True)
usim.info()

UniSim is storing a copy of the indexed data
if you are using large data corpus consider disable this behavior using store_data=False
[Indexer]
|-is_exact:True
|-use_tf_knn:True
|-store index data:True


In [22]:
e = usim.text.embed('lalal')  # warmup

In [25]:
usim.text.reset_index()

cnts = PerfCounters()
cnts.start('total')

cnts.start('indexing')
usim.text.batch_index(train_texts)
cnts.stop('indexing')

cnts.start('indexing2')
usim.text.batch_index(test_texts[:20])
cnts.stop('indexing2')

cnts.start('search')
dups = usim.text.batch_search(test_texts)
cnts.stop('search')
cnts.stop('total')
cnts.report()


gk 5 pk 5
-=[Timing counters]=-
+-----------+------------+
| name      |      value |
|-----------+------------|
| total     | 125.759    |
| indexing  | 120.557    |
| search    |   5.05722  |
| indexing2 |   0.143259 |
+-----------+------------+




In [30]:
iqps = len(train_texts) / cnts.get('indexing')
sqps = len(test_texts) / cnts.get('search')
print(f'indexing {round(iqps)} qps')
print(f'search {round(sqps)} qps')

indexing 71 qps
search 211 qps


In [34]:
wiki_time = (3_000_000 / iqps)
wiki_hour = wiki_time / 3600
print(f"{round(wiki_time)} sec, {round(wiki_hour, 1)}h")

42400 sec, 11.8h


In [26]:
# for d in dups:
#     if d.is_partial:
        
for d in dups:
    if d.num_global_matches:
        usim.viz.result(d)
        break

Query 0: "lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness ."
  idx  is_global      global_sim  is_partial      partial_sim  content
-----  -----------  ------------  ------------  -------------  ------------------------------
 8530  True                 1     True                   1     lovingly photographed in the m
  909  False                0.56  False                  0.56  culkin , who's in virtually ev
 3951  False                0.55  False                  0.55  baran is shockingly devoid of
 8209  False                0.54  False                  0.54  unfortunately , one hour photo
 3510  False                0.54  False                  0.54  if divine secrets of the ya-ya
